# Ranking dos Pilotos de 2025 com Dados

A ideia aqui neste notebook é realizar um rankeamento de pilotos com objetividade, baseado em dados, utilizando features e métodos de data science.

## Metodologia:

Utilizar features geradas com base nos dados de F1 e criarmos um algoritmo de normalização que vai trabalhar cada categoria de features e criar um score hipotético para cada piloto em cada categoria de features.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

# Explorando features criadas

Este é o primeiro notebook que usa o pipeline de geração de features, portanto, vale a pena, e também é bom pra abordar no vídeo posteriormente, analisar as features criadas em maior detalhe para garantir que está tudo ok e ver os destaques etc.

In [ ]:
# Caminhos para os arquivos
pace_path = '../../data/features/pace_features.csv'
perf_path = '../../data/features/performance_features.csv'
rel_path = '../../data/features/reliability_features.csv'

df_pace = pd.read_csv(pace_path)
df_perf = pd.read_csv(perf_path)
df_rel = pd.read_csv(rel_path)

## 1. Sanity Check: Pace Features

Verificando estrutura, nulos e distribuição básica das métricas de ritmo.

In [ ]:
df_pace.describe()

In [ ]:
df_pace

## 2. Sanity Check: Performance Features

Analisando dados de performance (podiums, wins, points, etc).

In [ ]:
display(df_perf.head())

print("\n--- Info ---")
df_perf.info()

print("\n--- Estatísticas Descritivas ---")
display(df_perf.describe())

print("\n--- Verificação de Nulos ---")
print(df_perf.isnull().sum()[df_perf.isnull().sum() > 0])

## 3. Sanity Check: Reliability Features

Verificando dados de confiabilidade (DNFs, falhas mecânicas).

In [ ]:
display(df_rel.head())

print("\n--- Info ---")
df_rel.info()

print("\n--- Estatísticas Descritivas ---")
display(df_rel.describe())

print("\n--- Verificação de Nulos ---")
print(df_rel.isnull().sum()[df_rel.isnull().sum() > 0])

## 4. Visualização de Distribuições

Uma rápida visualização para identificar outliers ou comportamentos estranhos nas principais métricas.

In [ ]:
# Exemplo: Distribuição de Pontos por Temporada (Performance)
plt.figure(figsize=(10, 5))
sns.histplot(df_perf['total_points'], kde=True)
plt.title('Distribuição de Total Points')
plt.show()

# Exemplo: Distribuição de Avg Lap Time Gap (Pace)
if 'avg_lap_time_gap' in df_pace.columns:
    plt.figure(figsize=(10, 5))
    sns.histplot(df_pace['avg_lap_time_gap'], kde=True)
    plt.title('Distribuição de Avg Lap Time Gap')
    plt.show()